<a href="https://colab.research.google.com/github/dennis-ancajima/ML-PUCP/blob/main/04_Modelo_Entrenamiento_17_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En el presente cuaderno vamos a realizar la experimentación de resultados propios considerando todas las características del conjunto de datos comparando el método de codificación Label Encoder vs Dummies aplicando diferentes métodos de clasificación para estimar los niveles de obesidad con un conjunto de datos  pre-procesado en el cuaderno: **03_Preprocesamiento.ipynb**. Y utilizando validación cruzada considerando (k=10)

 **Tabla de contenido:**

1. Cargar datos.
2. Modelado.

  2.1. Considerando la columna calculada: IMC y codificador: Dummies.

  2.2. Considerando la columna calculada: IMC y codificador: LabelEncoder.

  2.3. Sin considerar la columna calculada: IMC y codificador: dummies.

  2.4. Sin considerar la columna calculada: IMC y codificador: LabelEncoder.

4. Evaluación de Resultados



# 1. Cargar datos

En primero lugar, realizamos la importación de librerías y módulos necesarios para el análisis de datos y aprendizaje automático.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt #visualizaciones
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import seaborn as sns #Libreria para gráficar
from sklearn.model_selection import cross_val_score
from google.colab import drive

pd.set_option("display.max_columns", 50) #Controla la cantidad máxima de columnas que se mostrarán al imprimir un DataFrame.

%matplotlib inline
plt.style.use('ggplot')

Realizamos la lectura de los archivo CSV pre-procesados sobre el conjunto de datos "ObesityDataSet_raw_and_data_sinthetic.csv" y los almacenamos en la siguientes variables utilizando la función read_csv() de la biblioteca panda.

In [4]:
#Lectura de los dataset pre-procesados
archivo1 = "17c_dataset_imc_dummies.csv"
archivo2 = "17c_dataset_imc_labelenconder.csv"
archivo3 = "17c_dataset_sinimc_dummies.csv"
archivo4 = "17c_dataset_sinimc_labelenconder.csv"

#El método "read_csv" nos permite leer un excel
dataset_imc_dummies = pd.read_csv(archivo1)
dataset_imc_labelenconder = pd.read_csv(archivo2)
dataset_sinimc_dummies = pd.read_csv(archivo3)
dataset_sinimc_labelenconder = pd.read_csv(archivo4)

# 2. Modelado

## 2.1. Considerando la columna calculada: IMC y codificador: Dummies.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [5]:
dataset_imc_dummies.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,NObeyesdad,IMC,EsMujer,History_Family,FAVC_DU,SMOKE_DU,SCC_DU,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,21.0,1.62,64.0,2.0,3.0,2.0,0.0,1.0,Normal_Weight,24.3865,True,True,False,False,False,0,0,1,0,0,0,0,1,0,0,0,1,0
1,21.0,1.52,56.0,3.0,3.0,3.0,3.0,0.0,Normal_Weight,24.2382,True,True,False,True,True,0,0,1,0,0,0,1,0,0,0,0,1,0
2,23.0,1.80,77.0,2.0,3.0,2.0,2.0,1.0,Normal_Weight,23.7654,False,True,False,False,False,0,0,1,0,0,1,0,0,0,0,0,1,0
3,27.0,1.80,87.0,3.0,3.0,2.0,2.0,0.0,Overweight_Level_I,26.8519,False,False,False,False,False,0,0,1,0,0,1,0,0,0,0,0,0,1
4,22.0,1.78,89.8,2.0,1.0,2.0,0.0,0.0,Overweight_Level_II,28.3424,False,False,False,False,False,0,0,1,0,0,0,1,0,0,0,0,1,0


**División de datos de entrenamiento y test**

In [9]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_imc_dummies.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_imc_dummies['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [10]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [11]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 27)
Test shape: (627, 27)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [12]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [16]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8958904109589041
Varianza de SVM Cúbico:  0.013629964891871502
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.8947


Observamos un síntoma claro de sobreajuste (overfitting).

### Modelo: Random Forest con bagging

In [13]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9835616438356164
Varianza de Random Forest:  0.0076270744696301475
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9697


### Modelo: Redes neuronales

In [18]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9287671232876713
Varianza de Random Forest:  0.020132792406437075
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9155


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


**Conclusión**: Mejor modelo con Random Forest, con una exactitud en el conjunto de validación de **0.9697**

## 2.2. Considerando la columna calculada: IMC y codificador: LabelEncoder.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [28]:
dataset_imc_labelenconder.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,IMC
0,0,21.0,1.62,64.0,1,0,2.0,3.0,2,0,2.0,0,0.0,1.0,3,3,1,24.3865
1,0,21.0,1.52,56.0,1,0,3.0,3.0,2,1,3.0,1,3.0,0.0,2,3,1,24.2382
2,1,23.0,1.80,77.0,1,0,2.0,3.0,2,0,2.0,0,2.0,1.0,1,3,1,23.7654
3,1,27.0,1.80,87.0,0,0,3.0,3.0,2,0,2.0,0,2.0,0.0,1,4,5,26.8519
4,1,22.0,1.78,89.8,0,0,2.0,1.0,2,0,2.0,0,0.0,0.0,2,3,6,28.3424


**División de datos de entrenamiento y test**

In [29]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_imc_labelenconder.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_imc_labelenconder['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [30]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [31]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 17)
Test shape: (627, 17)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [32]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [33]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8965753424657533
Varianza de SVM Cúbico:  0.02175668517023094
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9139


Obtenemos una exactitud de **0.9139**

### Modelo: Random Forest con bagging

In [34]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.986986301369863
Varianza de Random Forest:  0.006461630912367524
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9617


### Modelo: Redes neuronales

In [35]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9260273972602739
Varianza de Random Forest:  0.014943441252925634
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9378


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


### Ensamble por votación manual

In [36]:
#Importación de bibliotecas y clasificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Crear  instancias para todos los siguientes clasificadores
estimators = [
     ('Random Forest', make_pipeline(StandardScaler(), RandomForestClassifier(oob_score = True, random_state=42))),
     ('SVM Cubic', make_pipeline(StandardScaler(),svm.SVC(kernel='poly', degree=3,C=10, gamma=50,probability=True))),

     ('árbol de decisión', DecisionTreeClassifier(max_depth=4)),
     ('k-NN', make_pipeline(StandardScaler(),
                            KNeighborsClassifier(n_neighbors=7) )),
     ('SVM de kernel gaussiano', make_pipeline(StandardScaler(),
                                               SVC(kernel='rbf', probability=True) ))
]

#Definición del ensamble
ensamble = VotingClassifier(estimators=estimators,
                        voting='soft', weights=[3,2,2, 1, 2])

#Ajuste del ensamble
ensamble.fit(X_train, y_train)

for (name, estimator) in ensamble.named_estimators_.items():
  print(f'Exactitud de {name: <25}: Entrenamiento {estimator.score(X_train, y_train):.03f} || Pruebas {estimator.score(X_val, y_val):.03f}',  )

print(f'Exactitud de {"ensamble": <25}: Entrenamiento {ensamble.score(X_train, y_train):.03f} || Validación {ensamble.score(X_val, y_val):.03f}' )

Exactitud de Random Forest            : Entrenamiento 1.000 || Pruebas 0.994
Exactitud de SVM Cubic                : Entrenamiento 1.000 || Pruebas 0.917
Exactitud de árbol de decisión        : Entrenamiento 0.977 || Pruebas 0.974
Exactitud de k-NN                     : Entrenamiento 0.846 || Pruebas 0.842
Exactitud de SVM de kernel gaussiano  : Entrenamiento 0.964 || Pruebas 0.914
Exactitud de ensamble                 : Entrenamiento 0.999 || Validación 0.984


**Conclusión**: Mejor modelo con Ensamble por votación manual, con una exactitud en el conjunto de validación de **0.984**

## 2.3. Sin considerar la columna calculada: IMC y codificador: dummies.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [37]:
dataset_sinimc_dummies.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,NObeyesdad,EsMujer,History_Family,FAVC_DU,SMOKE_DU,SCC_DU,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,21.0,1.62,64.0,2.0,3.0,2.0,0.0,1.0,Normal_Weight,True,True,False,False,False,0,0,1,0,0,0,0,1,0,0,0,1,0
1,21.0,1.52,56.0,3.0,3.0,3.0,3.0,0.0,Normal_Weight,True,True,False,True,True,0,0,1,0,0,0,1,0,0,0,0,1,0
2,23.0,1.80,77.0,2.0,3.0,2.0,2.0,1.0,Normal_Weight,False,True,False,False,False,0,0,1,0,0,1,0,0,0,0,0,1,0
3,27.0,1.80,87.0,3.0,3.0,2.0,2.0,0.0,Overweight_Level_I,False,False,False,False,False,0,0,1,0,0,1,0,0,0,0,0,0,1
4,22.0,1.78,89.8,2.0,1.0,2.0,0.0,0.0,Overweight_Level_II,False,False,False,False,False,0,0,1,0,0,0,1,0,0,0,0,1,0


**División de datos de entrenamiento y test**

In [38]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_sinimc_dummies.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_sinimc_dummies['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [39]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [40]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 26)
Test shape: (627, 26)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [41]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [42]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8691780821917808
Varianza de SVM Cúbico:  0.015753424657534244
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.8517


Observamos un síntoma claro de sobreajuste (overfitting).

### Modelo: Random Forest con bagging

In [43]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9417808219178083
Varianza de Random Forest:  0.019190309213865626
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9330


Con Random Forest se obtiene una exactitud en el conjunto de validación de **0.9330**

### Modelo: Redes neuronales

In [44]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9095890410958903
Varianza de Random Forest:  0.014626134591823707
Exactitud en el conjunto de entrenamiento: 0.9993
Exactitud en el conjunto de validación: 0.9043


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


**Conclusión**: Mejor modelo con Random Forest, con una exactitud en el conjunto de validación de **0.9330**

## 2.4. Sin considerar la columna calculada: IMC y codificador: LabelEncoder.

Veamos las primeras 5 filas del conjunto de datos almacenado en la variable dataset.

In [45]:
dataset_sinimc_labelenconder.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.0,1.62,64.0,1,0,2.0,3.0,2,0,2.0,0,0.0,1.0,3,3,1
1,0,21.0,1.52,56.0,1,0,3.0,3.0,2,1,3.0,1,3.0,0.0,2,3,1
2,1,23.0,1.80,77.0,1,0,2.0,3.0,2,0,2.0,0,2.0,1.0,1,3,1
3,1,27.0,1.80,87.0,0,0,3.0,3.0,2,0,2.0,0,2.0,0.0,1,4,5
4,1,22.0,1.78,89.8,0,0,2.0,1.0,2,0,2.0,0,0.0,0.0,2,3,6


**División de datos de entrenamiento y test**

In [46]:
#Separamos la data, en variables independientes (x) y dependientes (y), para poder entrenar el modelo de regresión
X_train_val = dataset_sinimc_labelenconder.drop(['NObeyesdad'], axis=1)
y_train_val = dataset_sinimc_labelenconder['NObeyesdad']

Debido al tamaño del conjunto de datos se decidió utilizar el 70\% de los datos para el entrenamiento y el 30\% para la validación.

In [47]:
# Dividimos la data en conjunto de "entrenamiento" y "prueba"
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [48]:
#Observamos el tamaño actual de los datos de entrenamiento y de validación.
print(f"Train shape: {X_train.shape}")
print(f"Test shape: {X_val.shape}")

Train shape: (1460, 16)
Test shape: (627, 16)


Decidimos utilizar el método **StandardScaler** para escalar los datos: siendo la media igual a 0 y la varianza igual a 1.

In [49]:
# Se crea un objeto de la clase StandardScaler de la librería scikit-learn.
scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train) #Escalar los datos de entrenamiento
X_val_scal = scaler.fit_transform(X_val) #Escalar los datos de prueba

### Modelo: SVM Cúbico

In [50]:
# Tomado de notebook trabajado en clase "SVM - Ejercicio.ipynb"
from sklearn import svm
#Importa la función accuracy_score del módulo sklearn.metrics, que se utilizará para calcular la precisión del modelo.
from sklearn.metrics import accuracy_score

# Crea una instancia del clasificador SVM utilizando el kernel cúbico.
model = svm.SVC(kernel='poly', degree=3,C=0.0316, gamma=1)

# Entrenamiento del clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#Realiza una validación cruzada de 10 divisiones utilizando el modelo SVM y los datos de entrenamiento escalados.
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)

#Calcula la media de las puntuaciones de rendimiento obtenidas en la validación cruzada.
cvForest = cv.mean()
# Calcula la desviación estándar de las puntuaciones de rendimiento obtenidas en la validación cruzada.
stdForest = cv.std()


print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de SVM Cúbico: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.8664383561643836
Varianza de SVM Cúbico:  0.019433234190563163
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.8804


Obtenemos una exactitud de **0.8804** (Overfitting)

### Modelo: Random Forest con bagging

In [51]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador RandomForest
model = RandomForestClassifier(oob_score = True, random_state=42)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10) #metodo para obtener la precisión
cvForest = cv.mean()
stdForest = cv.std()

print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

promedio Validación cruzada k=10:  0.9445205479452055
Varianza de Random Forest:  0.0171779947999787
Exactitud en el conjunto de entrenamiento: 1.0000
Exactitud en el conjunto de validación: 0.9346


### Modelo: Redes neuronales

In [52]:
#Importación de librerias
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Crear una instancia del clasificador MLPClassifier
model = MLPClassifier(random_state=1, max_iter=300)

# Entrenar el clasificador
model.fit(X_train_scal, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val_scal)

#metodo para obtener la precisión
cv = cross_val_score(estimator = model, X = X_train_scal, y = y_train, cv = 10)
cvForest = cv.mean()
stdForest = cv.std()

#Resultados
print("promedio Validación cruzada k=10: ", cvForest)
print('Varianza de Random Forest: ',stdForest)
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % model.score(X_train_scal, y_train))
print ('Exactitud en el conjunto de validación: %0.4f' % model.score(X_val_scal, y_val))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

promedio Validación cruzada k=10:  0.9102739726027398
Varianza de Random Forest:  0.018745112580005475
Exactitud en el conjunto de entrenamiento: 0.9993
Exactitud en el conjunto de validación: 0.9266


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


### Ensamble por votación manual

In [53]:
#Ejercicio considerado de la notebook desarrollado en clase.
#Importación de bibliotecas y clasificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Crear  instancias para todos los siguientes clasificadores
estimators = [
     ('Random Forest', make_pipeline(StandardScaler(), RandomForestClassifier(oob_score = True, random_state=42))),
     ('SVM Cubic', make_pipeline(StandardScaler(),svm.SVC(kernel='poly', degree=3,C=10, gamma=50,probability=True))),

     ('árbol de decisión', DecisionTreeClassifier(max_depth=4)),
     ('k-NN', make_pipeline(StandardScaler(),
                            KNeighborsClassifier(n_neighbors=7) )),
     ('SVM de kernel gaussiano', make_pipeline(StandardScaler(),
                                               SVC(kernel='rbf', probability=True) ))
]

#Definición del ensamble
ensamble = VotingClassifier(estimators=estimators,
                        voting='soft', weights=[3,2,2, 1, 2])

#Ajuste del ensamble
ensamble.fit(X_train, y_train)

for (name, estimator) in ensamble.named_estimators_.items():
  print(f'Exactitud de {name: <25}: Entrenamiento {estimator.score(X_train, y_train):.03f} || Pruebas {estimator.score(X_val, y_val):.03f}',  )

print(f'Exactitud de {"ensamble": <25}: Entrenamiento {ensamble.score(X_train, y_train):.03f} || Validación {ensamble.score(X_val, y_val):.03f}' )

Exactitud de Random Forest            : Entrenamiento 1.000 || Pruebas 0.955
Exactitud de SVM Cubic                : Entrenamiento 1.000 || Pruebas 0.869
Exactitud de árbol de decisión        : Entrenamiento 0.805 || Pruebas 0.788
Exactitud de k-NN                     : Entrenamiento 0.828 || Pruebas 0.813
Exactitud de SVM de kernel gaussiano  : Entrenamiento 0.944 || Pruebas 0.880
Exactitud de ensamble                 : Entrenamiento 0.997 || Validación 0.939


**Conclusión**: Mejor modelo con Ensamble por votación manual, con una exactitud en el conjunto de validación de **0.939**

# 3. Evaluación de Resultados

Se concluye que el mejor modelo tuvo una exactitud de **0.984** con el modelo Ensamble por votación manual, con una validación cruzada de 10 folders.